In [1]:
import os
os.chdir("../")

%pwd

'g:\\LLM Project\\FinBot_Using_LLM'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    local_dir : Path

In [3]:
from src.FinancialBot.constants import *
from src.FinancialBot.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            # self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                local_dir=config.local_dir,
          )
          return data_ingestion_config

In [5]:
import os 
import urllib.request as request
from src.FinancialBot import logger
from src.FinancialBot.utils.common import read_yaml
from sec_edgar_downloader import Downloader



[2024-05-04 01:00:36,850: INFO: limiter: Initializing default bucket(InMemoryBucket) with rates: [limit=10/1000]]


In [6]:
class DataIngestion:
    def __init__(self, company_name, email_address, save_location):
        self.company_name = company_name
        self.email_address = email_address
        self.save_location = save_location
        self.dl = Downloader(company_name, email_address, save_location)

    def download_10k_filings(self, ticker):
        download_status = {}
        for year in range(1999, 2023):
            try:
                self.dl.get("10-K", ticker, after=f"{year}-01-01", before=f"{year+1}-01-01")
                download_status[year] = "Success"
                logger.info(f"Successfully downloaded 10-K filings for {ticker} for year {year}")
            except Exception as e:
                download_status[year] = f"Failed: {str(e)}"
                logger.error(f"Failed to download 10-K filings for {ticker} for year {year}: {str(e)}")
        return download_status


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion("Apple Inc.", "skapaliya@gmail,com", data_ingestion_config.local_dir)
    data_ingestion.download_10k_filings("AAPL")
except Exception as e:
    logger.error(e)
    raise

[2024-05-04 01:00:36,906: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-04 01:00:36,908: INFO: common: created directory at: artifacts]
[2024-05-04 01:00:36,909: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-04 01:00:38,964: INFO: 354788097: Successfully downloaded 10-K filings for AAPL for year 1999]
[2024-05-04 01:00:39,987: INFO: 354788097: Successfully downloaded 10-K filings for AAPL for year 2000]
[2024-05-04 01:00:41,000: INFO: 354788097: Successfully downloaded 10-K filings for AAPL for year 2001]
[2024-05-04 01:00:42,241: INFO: 354788097: Successfully downloaded 10-K filings for AAPL for year 2002]
[2024-05-04 01:00:43,447: INFO: 354788097: Successfully downloaded 10-K filings for AAPL for year 2003]
[2024-05-04 01:00:44,754: INFO: 354788097: Successfully downloaded 10-K filings for AAPL for year 2004]
[2024-05-04 01:00:45,897: INFO: 354788097: Successfully downloaded 10-K filings for AAPL for year 2005]
[2024-05-04 01:00:47